In [38]:
# from predictNew import LinkedinScraper, PreprocessData, Predict
import pandas as pd

In [12]:
data = PreprocessData()

In [13]:
total_df = data.transform('data/dsjobs_training_culled.csv', 'data/ethan_profile.csv')

In [16]:
prediction = Predict(total_df)

In [19]:
recommended_posting = pd.DataFrame(total_df.iloc[1, :]).T

In [21]:
recommended_posting

,indices,jobs,labels,distances,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,...,4596,4597,4598,4599,4600,4601,4602,4603,4604,4605,4606,4607,4608,4609,4610,4611,4612,4613,4614,4615,4616,4617,4618,4619,4620,4621,4622,4623,4624,4625,4626,4627,4628,4629,4630,4631,4632,4633,4634,4635,4636,4637,4638,4639,4640,4641,4642,4643,4644,4645,4646,4647,4648,4649,4650,4651,4652,4653,4654,4655,4656,4657,4658,4659,4660,4661,4662,4663,4664,4665,4666,4667,4668,4669,4670,4671,4672,4673,4674,4675,4676,4677,4678,4679,4680,4681,4682,4683,4684,4685,4686,4687,4688,4689,4690,4691,4692,4693,4694,4695,4696,4697
1,14,Are you interested in working for one of the m...,0,0.833479,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0164118,0.0265164,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0244253,0,0,0,0,0,0,0.03136,0,0,0,0,0,0,0.058694,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0261232,0,0,0,0,0,0,0,0,0,0,0,0,0


In [22]:
original_index = int(recommended_posting['indices'])

In [23]:
original_index

14

In [24]:
label_index = total_df[total_df['indices'] == original_index].index[0]

In [25]:
label_index

1

In [49]:
import test

In [50]:
test.index

1

In [51]:
test.do_stuff()

In [52]:
test.index

2